In [ ]:
../julia-1.10.0/bin/julia --project=. --thread 32

In [ ]:
grad_k11 = tmp.grad_k11; prob = tmp.prob; v_prod = tmp.v_prod;
@time @tullio verbose=2 grad_k11[i, j, m, h] = prob[a, i, m]*v_prod[a, j, m, h] * (j!=i);
Grad_k11 = permutedims(grad_k11,(4,1,2,3)); Prob = prob; V_prod = permutedims(v_prod,(4,1,2,3));
@time @tullio verbose=2 Grad_k11[h, i, j, m] = Prob[a, i, m]*V_prod[h, a, j, m] * (j!=i);

grad_k11 = tmp.grad_k11; prob = tmp.prob; v_prod = tmp.v_prod;
@btime @tullio grad_k11[i, j, m, h] = prob[a, i, m]*v_prod[a, j, m, h] * (j!=i);
@btime @tullio grad_k11[i, j, m, h] = prob[a, i, m]*v_prod[a, j, m, h];
@btime @tullio grad_k11[i, j, m, h] = grad_k11[i, j, m, h] * (j!=i);





Grad_k11 = permutedims(grad_k11,(1,3,4,2)); Prob = permutedims(prob, (1,3,2)); V_prod = permutedims(v_prod,(1,3,4,2));
@time @tullio Grad_k11[i, m, h, j] = Prob[a, m, i]*V_prod[a, m, h, j] * (j!=i);



In [ ]:
#code for analytical gradient
using HopfieldDCA; using Tullio; import Flux: gradient;
filepath = "../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz"; H = 15; alg = HopPlmVar_gen(H, filepath;T=Float64); _w = alg.W ./ sum(alg.W); @tullio delta_j[a, j, m] := a == alg.Z[j, m] (a in 1:alg.q); tmp = Stg(alg);
@time gt = get_anal_grad(alg.K, alg.V, alg.Z, _w, tmp);
@time gk,gv = gradient((p1,p2)->get_loss_J(p1,p2, alg.Z, _w), alg.K, alg.V);
sum(gt.K[:].-gk[:])
sum(gt.V[:].-gv[:])

import BenchmarkTools: @btime
@btime gkt,gvt = get_anal_grad(alg.K, alg.V, alg.Z, _w, tmp);
@btime gk,gv = gradient((p1,p2)->get_loss_J(p1,p2, alg.Z, _w), alg.K, alg.V);
n_ep = 30; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer(alg, n_ep, η = η, λ = λ);
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer(alg, n_ep, η = η, λ = λ);
println("the end")


In [ ]:
structfile = "../DataAttentionDCA/data/PF00014/PF00014_struct.dat"
s = HopfieldDCA.score(m.K, m.V); PPV_h = HopfieldDCA.compute_PPV(s, structfile);

close("all"); plot(ppv_plm[1], label="Plm"); plot(PPV_h, label="HopPlm");legend();savefig("ciao.png");

